In [2]:
import tensorflow as tf
import keras
from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Add
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam
from keras.engine.network import Network
from keras import backend as K

from keras.initializers import glorot_normal,orthogonal

from keras.callbacks import EarlyStopping
import csv

from PIL import Image

import numpy as np
import time
import cv2

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# sess = tf.Session(config=config)
# K.set_session(sess)


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [3]:
#どんくらいで学習させるか関連x_test
num_sample=3752 #画像の総フレーム数
timesteps=30 #一回のLSTMに入れる値の数
camera=5 #カメラの数


#画像関連
channels=3
img_width=50
img_height=50

# LSTMなどで用いる値
n_hidden = 256    # 出力次元
epochs = 100      # エポック数
batch_size = 5   # バッチサイズ

In [4]:
class DataLoader:
    def __init__(self, camera_num, file_name, samples, timesteps, width, height):
        self.camera_num = camera_num
        self.file_name =file_name
        self.samples=samples
        self.timesteps=timesteps
        self.width=width
        self.height=height
        self.learnData=[]
        self.labelData=[]
        self.make_csvdata()
        self.make_selection()
    
    def make_selection(self):
        csv_selection=csv.reader(open(self.file_name+"video/mintime_optimize.csv", 'r'))
        one_hot = np.eye(self.camera_num)
        for row2 in csv_selection:
            self.labelData.append(one_hot[int(row2[1])])
    
    
    def make_csvdata(self):
        for i in range(0, self.camera_num):
            csv_selection=csv.reader(open(self.file_name+"video/head.csv", 'r'))
            img_list=[]
            for row2 in csv_selection:
                img_list.append([row2[i]])
            print(i)
            self.learnData.append(img_list)
        arr_multi = np.array(self.learnData)
        print(arr_multi.shape)
    
    
    def get_learndata(self, num):
        test=[]
        for i in range(0,self.samples-self.timesteps):
            tmp_list=[]
            for j in range(0, self.timesteps):
                tmp_list.append(self.learnData[num][i+j])
            test.append(tmp_list)
        tmp_list=np.array(test)
        tmp_list=tmp_list.astype(np.float)
                
        return tmp_list
    
    def get_labeldata(self):
        test=[]
        for i in range(self.timesteps,self.samples):
            test.append(self.labelData[i])
        tmp_list = np.array(test)
        tmp_list = tmp_list.astype(np.float)
        
        return tmp_list
    
    def show(self):
        print(self.get_learndata(1).shape)
        print(self.get_labeldata().shape)

In [5]:
data = DataLoader(camera, "./", num_sample, timesteps, img_width, img_height)
data.show()

0
1
2
3
4
(5, 3752, 1)
(3722, 30, 1)
(3722, 5)


In [6]:
class Resnet18 :
    ##
    ##　Resnet-18の作成
    ##
    def __init__(self, width, height, channels):
        self.img_rows=height
        self.img_cols=width
        self.img_channels=channels
    
    def rescell(self, data, filters, kernel_size, option=False):
        strides=(1,1)
        if option:
            strides=(2,2)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(data)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)
        
        data=Convolution2D(filters=int(x.shape[3]), kernel_size=(1,1), strides=strides, padding="same")(data)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=(1,1),padding="same")(x)
        x=BatchNormalization()(x)
        x=Add()([x,data])
        x=Activation('relu')(x)
        
        return x
    
    def create_resnet18(self):
        inputs=Input(shape=(self.img_rows, self.img_cols, self.img_channels))
        x=Convolution2D(64,(7,7), padding="same", input_shape=(self.img_rows, self.img_cols),activation="relu")(inputs)
        x=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        
        x=self.rescell(x,128,(3,3),True)
        x=self.rescell(x,128,(3,3))

        x=self.rescell(x,256,(3,3),True)
        x=self.rescell(x,256,(3,3))
        
        x=self.rescell(x,512,(3,3),True)
        x=self.rescell(x,512,(3,3))
        
        model=Model(inputs=inputs,outputs=[x])
        return model

    def get_resnet18(self):
        return self.create_resnet18()

In [7]:
class Prediction :
    def __init__(self, timesteps, n_hidden, n_out, width, height):
        self.maxlen = timesteps
        self.n_hidden = n_hidden
        self.n_out = n_out
        self.width=width
        self.height=height
        self.resnet = Resnet18(width, height, 3).get_resnet18()
        self.sharedLayer = self.create_sharedmodel()
    
    def create_sharedmodel(self):
        inputs = Input(shape=(self.maxlen, 1))
        
        predictions = LSTM(self.n_hidden, batch_input_shape = (None, self.maxlen, 1),
             kernel_initializer = glorot_normal(seed=20181020),
             recurrent_initializer = orthogonal(gain=1.0, seed=20181020), 
             dropout = 0.01, 
             recurrent_dropout = 0.01)(inputs)
        
        shared_layers = Model(inputs, predictions, name="shared_layers")
        
        return shared_layers
    
    def create_model(self):
        model_input1 = Input(shape=(self.maxlen, 1))
        model_input2 = Input(shape=(self.maxlen, 1))
        model_input3 = Input(shape=(self.maxlen, 1))
        model_input4 = Input(shape=(self.maxlen, 1))
        model_input5 = Input(shape=(self.maxlen, 1))
        
        mid_feature1 = self.sharedLayer(model_input1)
        mid_feature2 = self.sharedLayer(model_input2)
        mid_feature3 = self.sharedLayer(model_input3)
        mid_feature4 = self.sharedLayer(model_input4)
        mid_feature5 = self.sharedLayer(model_input5)
        
        merged_vector = keras.layers.concatenate([mid_feature1, mid_feature2, mid_feature3, mid_feature4, mid_feature5], axis=-1)
        
        mid_dense = Dense(self.n_hidden, activation='sigmoid')(merged_vector)
        predictions = Dense(self.n_out, activation='softmax')(mid_dense)
        
        model = Model(inputs=[model_input1, model_input2, model_input3, model_input4, model_input5], outputs=predictions)
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
        
        return model
    
    def train(self, x_train1, x_train2, x_train3, x_train4, x_train5, t_train, batch_size, epochs) :
        early_stopping = EarlyStopping(patience=5, verbose=1)
        model = self.create_model()
        hist = model.fit([x_train1, x_train2, x_train3, x_train4, x_train5], t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
                         callbacks = [early_stopping], validation_split = 0.3)
        return model, hist
    
    def train_keras(self, x_train1, x_train2, x_train3, x_train4, x_train5, t_train, batch_size, epochs) :
        early_stopping = EarlyStopping(patience=5, verbose=1)
        model = self.create_model()
        hist = model.fit([x_train1, x_train2, x_train3, x_train4, x_train5], t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
                         callbacks = [early_stopping], validation_split = 0.3)
        return model, hist

In [8]:
# モデル定義
pred = Prediction(timesteps, n_hidden, camera, img_width, img_height)
model = pred.create_model()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 30, 1)        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 30, 1)        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 30, 1)        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 30, 1)        0                                            
__________________________________________________________________________________________________
input_7 (I

In [9]:
# 学習
model , hist = pred.train(data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4), 
                                data.get_labeldata(), batch_size, epochs)
# テスト
score = model.evaluate([data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4)], 
                                data.get_labeldata(), batch_size = batch_size, verbose = 1)
print("score:", score)

pre = [[0 for i in range(5)] for j in range(5)]
# 正答率集計
preds = model.predict([data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4)])
correct = 0

f = open("./video/result_li.csv", 'w')
writer = csv.writer(f)
for i in range(0,timesteps):
    writer.writerow([0])
        
for i in range(len(preds)):
    pred_result = np.argmax(preds[i,:])
    tar = np.argmax(data.get_labeldata()[i,:])
    writer.writerow([pred_result])
    pre[pred_result][tar]+=1
    if pred_result == tar :
        correct += 1

print("正答率:", 1.0 * correct / len(preds))

Train on 2605 samples, validate on 1117 samples
Epoch 1/100
2605/2605 [==============================] - 134s 52ms/step - loss: 0.3817 - acc: 0.8385 - val_loss: 0.6266 - val_acc: 0.6788
Epoch 2/100
2605/2605 [==============================] - 133s 51ms/step - loss: 0.3776 - acc: 0.8425 - val_loss: 0.6025 - val_acc: 0.6788
Epoch 3/100
2605/2605 [==============================] - 124s 47ms/step - loss: 0.3779 - acc: 0.8429 - val_loss: 0.6304 - val_acc: 0.6788
Epoch 4/100
2605/2605 [==============================] - 131s 50ms/step - loss: 0.3777 - acc: 0.8438 - val_loss: 0.5798 - val_acc: 0.6788
Epoch 5/100
2605/2605 [==============================] - 133s 51ms/step - loss: 0.3779 - acc: 0.8434 - val_loss: 0.6499 - val_acc: 0.6788
Epoch 6/100
2605/2605 [==============================] - 136s 52ms/step - loss: 0.3778 - acc: 0.8433 - val_loss: 0.6230 - val_acc: 0.6788
Epoch 7/100
2605/2605 [==============================] - 134s 52ms/step - loss: 0.3767 - acc: 0.8441 - val_loss: 0.5242 - va

In [ ]:
a=[0,0,0,0,0]
for()